In [ ]:
!pip install torch torchvision pandas scikit-learn tqdm

In [1]:

import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
import zipfile

zip_path = "/content/drive/MyDrive/images2.zip"
extract_path = "/content"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:

class FitzpatrickDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, f"{row['md5hash']}.jpg")
        label = int(row['fitzpatrick_scale']) - 1  # 0-indexed classes
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import platform


def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=10):
    print(f"Starting training with {len(dataloaders['train'])} training batches")
    print(f"Training dataset size: {len(dataloaders['train'].dataset)}")
    print(f"Validation dataset size: {len(dataloaders['val'].dataset)}")

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        epoch_start_time = time.time()

        model.train()
        running_loss = 0.0
        batch_count = 0

        for batch_idx, (inputs, labels) in enumerate(tqdm(dataloaders['train'], desc="Training")):
            batch_start_time = time.time()

            # This snippet was added for debugging model taking too mch tie so to ensure it is working
            if batch_idx % 10 == 0:
                print(f"\nBatch {batch_idx}/{len(dataloaders['train'])}")
                print(f"Input shape: {inputs.shape}")
                print(f"Labels shape: {labels.shape}")

            # Data loading time
            data_load_time = time.time()
            inputs = inputs.to(device)
            labels = labels.to(device)
            data_transfer_time = time.time() - data_load_time

            # Forward pass time
            forward_start = time.time()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            forward_time = time.time() - forward_start

            # Backward pass time
            backward_start = time.time()
            loss.backward()
            optimizer.step()
            backward_time = time.time() - backward_start

            running_loss += loss.item() * inputs.size(0)
            batch_count += 1

            total_batch_time = time.time() - batch_start_time

            # For helping in debugging
            if batch_idx < 5 or batch_idx % 50 == 0:
                print(f"\nBatch {batch_idx} timing:")
                print(f"  Data transfer: {data_transfer_time:.3f}s")
                print(f"  Forward pass: {forward_time:.3f}s")
                print(f"  Backward pass: {backward_time:.3f}s")
                print(f"  Total batch: {total_batch_time:.3f}s")
                print(f"  Loss: {loss.item():.4f}")


        if batch_count > 0:
            epoch_loss = running_loss / len(dataloaders['train'].dataset)
            epoch_time = time.time() - epoch_start_time
            print(f"\nEpoch {epoch + 1} completed in {epoch_time:.1f} seconds")
            print(f"Training Loss: {epoch_loss:.4f}")
            print(f"Average time per batch: {epoch_time/batch_count:.3f}s")

            # Validation
            print("Starting validation...")
            val_start_time = time.time()
            model.eval()
            all_preds = []
            all_labels = []

            with torch.no_grad():
                for val_batch_idx, (inputs, labels) in enumerate(dataloaders['val']):
                    if val_batch_idx % 20 == 0:
                        print(f"Validation batch {val_batch_idx}/{len(dataloaders['val'])}")

                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            val_time = time.time() - val_start_time
            print(f"Validation completed in {val_time:.1f} seconds")
            print("Validation Metrics:")
            print(classification_report(all_labels, all_preds, digits=3, zero_division=0))
        else:
            print("Training stopped due to performance issues")
            break

    return model

def main():
    print("=== DEBUG INFORMATION ===")
    print(f"Platform: {platform.system()}")
    print(f"PyTorch version: {torch.__version__}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    if torch.cuda.is_available():
        print(f"CUDA device: {torch.cuda.get_device_name()}")
        print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    print("\n=== LOADING DATA ===")
    data_start_time = time.time()

    df = pd.read_csv("fitzpatrick17k.csv")
    df['image_path'] = df['md5hash'].apply(lambda x: f"images/{x}.jpg")
    df= df[df['image_path'].apply(lambda x: os.path.exists(x))]
    print(f"Total images found: {len(df)}")

    print(f"Original dataset size: {len(df)}")

    df = df[df['fitzpatrick_scale'].isin([1, 2, 3, 4, 5, 6])]
    print(f"Filtered dataset size: {len(df)}")
    print(f"Class distribution:\n{df['fitzpatrick_scale'].value_counts().sort_index()}")

    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['fitzpatrick_scale'], random_state=42)
    print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    image_dir = "images"
    print(f"Image directory: {image_dir}")

    print("\n=== CREATING DATASETS ===")
    dataset_start_time = time.time()

    train_dataset = FitzpatrickDataset(train_df, image_dir, transform)
    val_dataset = FitzpatrickDataset(val_df, image_dir, transform)

    dataset_time = time.time() - dataset_start_time
    print(f"Dataset creation took: {dataset_time:.3f} seconds")

    # Test loading a single sample
    print("\n=== TESTING SINGLE SAMPLE ===")
    try:
        sample_start = time.time()
        sample_data = train_dataset[0]
        sample_time = time.time() - sample_start
        print(f"Single sample load time: {sample_time:.3f} seconds")
        print(f"Sample image shape: {sample_data[0].shape}")
        print(f"Sample label: {sample_data[1]}")
    except Exception as e:
        print(f"ERROR loading single sample: {e}")
        return

    # Adjust num_workers based on platform
    num_workers = 0 if platform.system() == "Windows" else 2
    print(f"\nUsing num_workers: {num_workers}")

    # Start with smaller batch size for debugging
    batch_size = 16  # Reduced from 32
    print(f"Using batch_size: {batch_size}")

    dataloaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers),
        'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    }

    print("\n=== TESTING DATALOADER ===")
    try:
        dataloader_start = time.time()
        for i, (inputs, labels) in enumerate(dataloaders['train']):
            dataloader_time = time.time() - dataloader_start
            print(f"First batch load time: {dataloader_time:.3f} seconds")
            print(f"Batch input shape: {inputs.shape}")
            print(f"Batch labels shape: {labels.shape}")
            break
    except Exception as e:
        print(f"ERROR with dataloader: {e}")
        return

    print("\n=== CREATING MODEL ===")
    model_start_time = time.time()

    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

    num_features = model.classifier[1].in_features
    print(f"Original classifier features: {num_features}")

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(num_features, 6)
    )
    print(f"Modified classifier output features: 6")

    model = model.to(device)
    model_time = time.time() - model_start_time
    print(f"Model creation and transfer took: {model_time:.3f} seconds")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

    data_time = time.time() - data_start_time
    print(f"\nTotal setup time: {data_time:.1f} seconds")

    print("\n=== STARTING TRAINING ===")
    model = train_model(model, dataloaders, criterion, optimizer, device, num_epochs=5)

    torch.save(model.state_dict(), "fitzpatrick_skin_model.pth")
    print("Model saved as fitzpatrick_skin_model.pth")


if __name__ == "__main__":
    main()

=== DEBUG INFORMATION ===
Platform: Linux
PyTorch version: 2.6.0+cu124
Using device: cpu

=== LOADING DATA ===
Total images found: 16518
Original dataset size: 16518
Filtered dataset size: 15956
Class distribution:
fitzpatrick_scale
1    2941
2    4795
3    3294
4    2771
5    1527
6     628
Name: count, dtype: int64
Train size: 12764, Validation size: 3192
Image directory: images

=== CREATING DATASETS ===
Dataset creation took: 0.000 seconds

=== TESTING SINGLE SAMPLE ===
Single sample load time: 0.057 seconds
Sample image shape: torch.Size([3, 224, 224])
Sample label: 2

Using num_workers: 2
Using batch_size: 16

=== TESTING DATALOADER ===
First batch load time: 0.327 seconds
Batch input shape: torch.Size([16, 3, 224, 224])
Batch labels shape: torch.Size([16])

=== CREATING MODEL ===


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 125MB/s] 


Original classifier features: 1280
Modified classifier output features: 6
Model creation and transfer took: 0.426 seconds
Total parameters: 4,015,234
Trainable parameters: 4,015,234

Total setup time: 1.2 seconds

=== STARTING TRAINING ===
Starting training with 798 training batches
Training dataset size: 12764
Validation dataset size: 3192

Epoch 1/5


Training:   0%|          | 0/798 [00:00<?, ?it/s]


Batch 0/798
Input shape: torch.Size([16, 3, 224, 224])
Labels shape: torch.Size([16])


...

Training:  99%|█████████▉| 790/798 [45:25<00:26,  3.28s/it]


Batch 790/798
Input shape: torch.Size([16, 3, 224, 224])
Labels shape: torch.Size([16])


Training: 100%|██████████| 798/798 [45:52<00:00,  3.45s/it]


Epoch 5 completed in 2752.6 seconds
Training Loss: 0.7928
Average time per batch: 3.449s
Starting validation...


Validation batch 0/200
Validation batch 20/200
Validation batch 40/200
Validation batch 60/200
Validation batch 80/200
Validation batch 100/200
Validation batch 120/200
Validation batch 140/200
Validation batch 160/200
Validation batch 180/200
Validation completed in 172.5 seconds
Validation Metrics:
              precision    recall  f1-score   support

           0      0.565     0.473     0.515       588
           1      0.482     0.489     0.486       959
           2      0.346     0.372     0.358       659
           3      0.431     0.478     0.453       554
           4      0.467     0.395     0.428       306
           5      0.510     0.587     0.546       126

    accuracy                          0.455      3192
   macro avg      0.467     0.466     0.464      3192
weighted avg      0.460     0.455     0.456      3192

Model saved as fitzpatrick_skin_model.pth


In [4]:
import zipfile

zip_path = "/content/drive/MyDrive/images2.zip"
extract_path = "/content"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [2]:
# THis model is developed using mobilenet to see if acuracy improves

import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [10]:
import pandas as pd
import os
import shutil
from collections import Counter

df = pd.read_csv("fitzpatrick17k.csv")

df = df[df['fitzpatrick_scale'].notna()]
df['fitzpatrick_scale'] = df['fitzpatrick_scale'].astype(int)
df['filename'] = df['md5hash'] + '.jpg'
df['image_path'] = df['md5hash'].apply(lambda x: os.path.join("images", f"{x}.jpg"))

df = df[df['image_path'].apply(lambda x: os.path.exists(x))]
print(f" Total valid images found: {len(df)}")

df = df[df['fitzpatrick_scale'].isin([1, 2, 3, 4, 5, 6])]

scale_counts = Counter(df['fitzpatrick_scale'])
print("\n Image count per Fitzpatrick scale:")
for scale in sorted(scale_counts.keys()):
    print(f"Scale {scale}: {scale_counts[scale]} images")

output_dir = "images_by_scale"
for scale in range(1, 7):
    os.makedirs(os.path.join(output_dir, f"scale_{scale}"), exist_ok=True)

copy_errors = 0
for _, row in df.iterrows():
    scale = row['fitzpatrick_scale']
    src = row['image_path']
    dst = os.path.join(output_dir, f"scale_{scale}", row['filename'])

    try:
        shutil.copy2(src, dst)
    except Exception as e:
        print(f" Failed to copy {row['filename']}: {e}")
        copy_errors += 1

print(f"\n All valid images copied by scale. Errors: {copy_errors}")


 Total valid images found: 16518

 Image count per Fitzpatrick scale:
Scale 1: 2941 images
Scale 2: 4795 images
Scale 3: 3294 images
Scale 4: 2771 images
Scale 5: 1527 images
Scale 6: 628 images

 All valid images copied by scale. Errors: 0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os
import shutil
import random
from glob import glob
from tqdm import tqdm

random.seed(42)

source_root = 'images_by_scale'
output_root = 'dataset'

splits = ['train', 'val', 'test']
for split in splits:
    for scale in range(1, 7):
        os.makedirs(os.path.join(output_root, split, f'scale_{scale}'), exist_ok=True)

# Split logic: 70% train, 15% val, 15% test
for scale in range(1, 7):
    class_path = os.path.join(source_root, f'scale_{scale}')
    all_images = glob(os.path.join(class_path, '*.jpg'))
    random.shuffle(all_images)

    n_total = len(all_images)
    n_train = int(0.7 * n_total)
    n_val = int(0.15 * n_total)

    train_imgs = all_images[:n_train]
    val_imgs = all_images[n_train:n_train + n_val]
    test_imgs = all_images[n_train + n_val:]

    for img in tqdm(train_imgs, desc=f'Copying scale {scale} to train'):
        shutil.copy2(img, os.path.join(output_root, 'train', f'scale_{scale}'))

    for img in tqdm(val_imgs, desc=f'Copying scale {scale} to val'):
        shutil.copy2(img, os.path.join(output_root, 'val', f'scale_{scale}'))

    for img in tqdm(test_imgs, desc=f'Copying scale {scale} to test'):
        shutil.copy2(img, os.path.join(output_root, 'test', f'scale_{scale}'))


Copying scale 1 to train:   0%|          | 0/2058 [00:00<?, ?it/s]

Copying scale 6 to test: 100%|██████████| 95/95 [00:01<00:00, 82.87it/s]


In [6]:
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# import os

# IMG_SIZE = 224
# BATCH_SIZE = 32
# NUM_CLASSES = 6
# EPOCHS = 30

# base_dir = "dataset"

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     horizontal_flip=True,
#     rotation_range=20,
#     zoom_range=0.2
# )

# val_datagen = ImageDataGenerator(rescale=1./255)

# train_gen = train_datagen.flow_from_directory(
#     os.path.join(base_dir, "train"),
#     target_size=(IMG_SIZE, IMG_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )

# val_gen = val_datagen.flow_from_directory(
#     os.path.join(base_dir, "val"),
#     target_size=(IMG_SIZE, IMG_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )

# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
# base_model.trainable = False

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.3)(x)
# output = Dense(NUM_CLASSES, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=output)

# model.compile(
#     optimizer=Adam(learning_rate=0.0001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# history = model.fit(
#     train_gen,
#     epochs=EPOCHS,
#     validation_data=val_gen
# )

# model.save("mobilenetv2_fitzpatrick_model.h5")


In [7]:
# test_gen = val_datagen.flow_from_directory(
#     os.path.join(base_dir, "test"),
#     target_size=(IMG_SIZE, IMG_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical',
#     shuffle=False
# )

# test_loss, test_acc = model.evaluate(test_gen)
# print(f"Test Accuracy: {test_acc:.4f}")

In [12]:

# Tryng augmentation to see if performance improves
import os

def count_images_in_dir(dir_path):
    class_counts = {}
    for class_name in sorted(os.listdir(dir_path)):
        class_path = os.path.join(dir_path, class_name)
        if os.path.isdir(class_path):
            count = len([
                f for f in os.listdir(class_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ])
            class_counts[class_name] = count
    return class_counts

base_dir = 'dataset'
splits = ['train', 'val', 'test']

for split in splits:
    print(f"\n Distribution in {split.upper()} folder:")
    counts = count_images_in_dir(os.path.join(base_dir, split))
    for cls, count in counts.items():
        print(f"  {cls}: {count} images")



 Distribution in TRAIN folder:
  scale_1: 2058 images
  scale_2: 3356 images
  scale_3: 2305 images
  scale_4: 1939 images
  scale_5: 1068 images
  scale_6: 439 images

 Distribution in VAL folder:
  scale_1: 441 images
  scale_2: 719 images
  scale_3: 494 images
  scale_4: 415 images
  scale_5: 229 images
  scale_6: 94 images

 Distribution in TEST folder:
  scale_1: 442 images
  scale_2: 720 images
  scale_3: 495 images
  scale_4: 417 images
  scale_5: 230 images
  scale_6: 95 images


In [13]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import numpy as np
from tqdm import tqdm

train_dir = 'dataset/train'
target_count = 3356  # Match to scale_2 count

augmentor = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

for scale in sorted(os.listdir(train_dir)):
    scale_path = os.path.join(train_dir, scale)
    images = [
        os.path.join(scale_path, f)
        for f in os.listdir(scale_path)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    current_count = len(images)
    needed = target_count - current_count
    print(f"\n Augmenting {scale}: {current_count} ➝ {target_count} (need {needed})")

    if needed <= 0:
        continue

    i = 0
    while i < needed:
        img_path = random.choice(images)
        img = load_img(img_path)
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)

        aug_iter = augmentor.flow(x, batch_size=1)
        for _ in range(min(needed - i, 1)):
            aug_img = next(aug_iter)[0].astype('uint8')
            new_img = array_to_img(aug_img)
            new_img_path = os.path.join(scale_path, f"aug_{i:04d}.jpg")
            new_img.save(new_img_path)
            i += 1



 Augmenting scale_1: 2058 ➝ 3356 (need 1298)

 Augmenting scale_2: 3356 ➝ 3356 (need 0)

 Augmenting scale_3: 2305 ➝ 3356 (need 1051)

 Augmenting scale_4: 1939 ➝ 3356 (need 1417)

 Augmenting scale_5: 1068 ➝ 3356 (need 2288)

 Augmenting scale_6: 439 ➝ 3356 (need 2917)


In [14]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import numpy as np
from tqdm import tqdm

train_dir = 'dataset/val'
target_count = 719

augmentor = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

for scale in sorted(os.listdir(train_dir)):
    scale_path = os.path.join(train_dir, scale)
    images = [
        os.path.join(scale_path, f)
        for f in os.listdir(scale_path)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    current_count = len(images)
    needed = target_count - current_count
    print(f"\n Augmenting {scale}: {current_count} ➝ {target_count} (need {needed})")

    if needed <= 0:
        continue

    i = 0
    while i < needed:
        img_path = random.choice(images)
        img = load_img(img_path)
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)

        aug_iter = augmentor.flow(x, batch_size=1)
        for _ in range(min(needed - i, 1)):
            aug_img = next(aug_iter)[0].astype('uint8')
            new_img = array_to_img(aug_img)
            new_img_path = os.path.join(scale_path, f"aug_{i:04d}.jpg")
            new_img.save(new_img_path)
            i += 1



 Augmenting scale_1: 441 ➝ 719 (need 278)

 Augmenting scale_2: 719 ➝ 719 (need 0)

 Augmenting scale_3: 494 ➝ 719 (need 225)

 Augmenting scale_4: 415 ➝ 719 (need 304)

 Augmenting scale_5: 229 ➝ 719 (need 490)

 Augmenting scale_6: 94 ➝ 719 (need 625)


In [15]:
import os

def count_images_in_dir(dir_path):
    class_counts = {}
    for class_name in sorted(os.listdir(dir_path)):
        class_path = os.path.join(dir_path, class_name)
        if os.path.isdir(class_path):
            count = len([
                f for f in os.listdir(class_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ])
            class_counts[class_name] = count
    return class_counts

base_dir = 'dataset'
splits = ['train', 'val', 'test']

for split in splits:
    print(f"\n Distribution in {split.upper()} folder:")
    counts = count_images_in_dir(os.path.join(base_dir, split))
    for cls, count in counts.items():
        print(f"  {cls}: {count} images")



 Distribution in TRAIN folder:
  scale_1: 3356 images
  scale_2: 3356 images
  scale_3: 3356 images
  scale_4: 3356 images
  scale_5: 3356 images
  scale_6: 3356 images

 Distribution in VAL folder:
  scale_1: 719 images
  scale_2: 719 images
  scale_3: 719 images
  scale_4: 719 images
  scale_5: 719 images
  scale_6: 719 images

 Distribution in TEST folder:
  scale_1: 442 images
  scale_2: 720 images
  scale_3: 495 images
  scale_4: 417 images
  scale_5: 230 images
  scale_6: 95 images


In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 6
EPOCHS = 50

base_dir = "dataset"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,      
    height_shift_range=0.2,
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(base_dir, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(base_dir, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# output = Dense(NUM_CLASSES, activation='softmax')(x)

# adding more layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        "mobilenetv2_fitzpatrick_best2.h5",
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks
)

model.save("mobilenetv2_fitzpatrick_final2.h5")


Found 20136 images belonging to 6 classes.
Found 4314 images belonging to 6 classes.
Epoch 1/50
630/630 [==============================] - ETA: 0s - loss: 1.6015 - accuracy: 0.3052
Epoch 1: val_accuracy improved from -inf to 0.39036, saving model to mobilenetv2_fitzpatrick_best2.h5
630/630 [==============================] - 266s 419ms/step - loss: 1.6015 - accuracy: 0.3052 - val_loss: 1.4498 - val_accuracy: 0.3904
Epoch 2/50
630/630 [==============================] - ETA: 0s - loss: 1.4857 - accuracy: 0.3507
Epoch 2: val_accuracy improved from 0.39036 to 0.39685, saving model to mobilenetv2_fitzpatrick_best2.h5
630/630 [==============================] - 275s 436ms/step - loss: 1.4857 - accuracy: 0.3507 - val_loss: 1.4417 - val_accuracy: 0.3968
Epoch 3/50
630/630 [==============================] - ETA: 0s - loss: 1.4558 - accuracy: 0.3683
Epoch 3: val_accuracy improved from 0.39685 to 0.40125, saving model to mobilenetv2_fitzpatrick_best2.h5
630/630 [==============================] - 30

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

IMG_SIZE = 224
BATCH_SIZE = 50
NUM_CLASSES = 6
EPOCHS = 50

base_dir = "dataset"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,      
    height_shift_range=0.2,
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(base_dir, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(base_dir, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# output = Dense(NUM_CLASSES, activation='softmax')(x)

# adding more layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)

output = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        "mobilenetv2_fitzpatrick_best2.h5",
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks
)

model.save("mobilenetv2_fitzpatrick_final2.h5")


Found 20136 images belonging to 6 classes.
Found 4314 images belonging to 6 classes.
Epoch 1/50
403/403 [==============================] - ETA: 0s - loss: 1.6674 - accuracy: 0.2694
Epoch 1: val_accuracy improved from -inf to 0.33380, saving model to mobilenetv2_fitzpatrick_best2.h5
403/403 [==============================] - 274s 675ms/step - loss: 1.6674 - accuracy: 0.2694 - val_loss: 1.5397 - val_accuracy: 0.3338
Epoch 2/50
403/403 [==============================] - ETA: 0s - loss: 1.5476 - accuracy: 0.3140
Epoch 2: val_accuracy improved from 0.33380 to 0.36602, saving model to mobilenetv2_fitzpatrick_best2.h5
403/403 [==============================] - 282s 698ms/step - loss: 1.5476 - accuracy: 0.3140 - val_loss: 1.4876 - val_accuracy: 0.3660
Epoch 3/50
403/403 [==============================] - ETA: 0s - loss: 1.5057 - accuracy: 0.3363
Epoch 3: val_accuracy did not improve from 0.36602
403/403 [==============================] - 257s 638ms/step - loss: 1.5057 - accuracy: 0.3363 - val_

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

IMG_SIZE = 224
BATCH_SIZE = 50
NUM_CLASSES = 6
EPOCHS = 50

base_dir = "dataset"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,      
    height_shift_range=0.2,
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(base_dir, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(base_dir, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# output = Dense(NUM_CLASSES, activation='softmax')(x)

# adding more layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)
# x = Dropout(0.2)(x)

output = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        "mobilenetv2_fitzpatrick_best2.h5",
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks
)

model.save("mobilenetv2_fitzpatrick_final2.h5")
